In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor

import geopandas as gpd
import joblib
import numpy as np
import odc.geo  # noqa: F401
from odc.stac import load
from planetary_computer import sign_url
from pystac_client import Client
from shapely import geometry
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

from utils import predict_xr

import pandas as pd

In [ ]:
%reload_ext autoreload
%autoreload 2

## Find and load S2 data

Load data and set up your array to use for prediction

In [ ]:
fiji_bbox = [177.14, -18.41, 179.80, -16.01]
fiji_bbox_geometry = geometry.box(*fiji_bbox)

fiji_gdf = gpd.GeoDataFrame({'geometry': [fiji_bbox_geometry]}, crs='EPSG:4326')
fiji_gdf.explore()

In [ ]:
# Configure some things up front
chunks = dict(x=2048, y=2048)
datetime = "2023"

In [ ]:
dep_client = Client.open("https://stac.staging.digitalearthpacific.org")

collection = "dep_s2_geomad"

items = list(dep_client.search(
    collections=[collection],
    bbox=fiji_bbox,
    datetime=datetime
).items())

print(f"Found {len(items)} items")

In [ ]:
# S2 bands https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a

bands = [
    "B02",
    "B03",
    "B04",
    "B05",
    "B06",
    "B07",
    "B08",
    "B8A",
    "B11",
    "B12",
    "emad",
    "bcmad",
    "smad",
]

data = load(
    items,
    bbox=fiji_bbox,
    measurements=bands,
    resolution=10,
    chunks=chunks,
).squeeze("time")

data

In [ ]:
# Add more indices here...

# Incorporate NDVI (Normalised Difference Vegetation Index) = (NIR-red)/(NIR+red)
data["ndvi"] = (data["B08"] - data["B04"]) / (data["B08"] + data["B04"])

# Incorporate MNDWI (Mean Normalised Difference Water Index) = (Green – SWIR) / (Green + SWIR)
data["mndwi"] = (data["B03"] - data["B12"]) / (data["B03"] + data["B12"])

# Incorporate EVI (Enhanced Vegetation Index) = 2.5NIR−RED(NIR+6RED−7.5BLUE)+1
data["evi"] = (2.5*(data["B08"] - data["B04"]))*((data["B08"] + (6*(data["B04"]) - (7.5*(data["B02"])))))+1

# Incorporate SAVI (Standard Vegetation Index) = (800nm−670nm) / (800nm+670nm+L(1+L)) # where L = 0.5
data["savi"] = (data["B07"] - data["B04"]) / (data["B07"] + data["B04"] + 0.5*(1 + 0.5))

# Incorporate BSI (Bare Soil Index) = ((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2)) # https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/barren_soil/
data["bsi"] = ((data["B11"] + data["B04"]) - (data["B08"] + data["B02"])) / ((data["B11"] + data["B04"]) + (data["B08"] + data["B02"])) 

# Incorporate NDMI (Normalised Difference Moisture Index) # https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndmi/
data["ndmi"] = ((data["B08"]) - (data["B11"])) / ((data["B08"]) + (data["B11"]))

# Incorporate NDBI (Normalised Difference Built-up Index) (B06 - B05) / (B06 + B05); # - built up ratio of vegetation to paved surface - let BU = (ndvi - ndbi) - https://custom-scripts.sentinel-hub.com/custom-scripts/landsat-8/built_up_index/
data["ndbi"] = ((data["B06"]) - (data["B05"])) / ((data["B06"]) + (data["B05"]))

data

In [ ]:
# Add in the DEM from the MSPC https://planetarycomputer.microsoft.com/dataset/cop-dem-glo-30

# Authorised access to MPC data.
# TODO: rotate key and grab it from an actual environment variable
os.environ["PC_SDK_SUBSCRIPTION_KEY"] = "84162f5502174b1b838239e74a44898d"

mspc_client = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")

# Get a pystac client for the MSPC
items_dem = list(mspc_client.search(collections=["cop-dem-glo-30"], bbox=fiji_bbox).items())
print(len(items_dem))

In [ ]:
data_dem = load(
    items_dem,
    chunks=chunks,
    groupby="solar_day",
    like=data,
    patch_url=sign_url
)

data_dem = data_dem.where(data_dem != -32768).rename({"data":"elevation"}).squeeze("time")

data_dem

In [ ]:
# Merge DEM with S2 data
data_s2_dem = data.update(data_dem)
data_s2_dem

## Find and load S1 data

Load data and set up your array to use for prediction

In [ ]:
# Add S-2 data from DEP
items_s1 = list(dep_client.search(collections=["dep_s1_mosaic"], bbox=fiji_bbox, datetime=datetime).items())
print(f"Found {len(items_s1)} items")

In [ ]:
data_s1 = load(
    items_s1,
    bbox=fiji_bbox,
    resolution=10,
    chunks=chunks,
    measurements=["mean_vv", "mean_vh"]
).squeeze("time")
data_s1["mean_vv_vh"] = (data_s1["mean_vv"]) / (data_s1["mean_vh"])

data_s1

## Merge together all data into an array

Merge all collated data and set up your array to use for prediction

In [ ]:
merged = data_s2_dem.update(data_s1)
merged

## Train and predict

When you change your training data, you can re-train and predict here.

In [ ]:
training_file = "training_data/MRD_dissagregated_25.geojson"

tdata = gpd.read_file(training_file, bbox=fiji_bbox_geometry)
tdata.explore()

In [ ]:
# Use Jesse's way

# Get values for each of the image bands at each of the points.
pts_proj = tdata.to_crs(merged.odc.crs)

# a DataArray with x & y coords
pts_da = pts_proj.assign(x=pts_proj.geometry.x, y=pts_proj.geometry.y).to_xarray()

# a dataframe or series (for a single point)
pt_values_i = (
    merged.sel(pts_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)

if isinstance(pt_values_i, pd.Series):
    pt_values_i = pt_values_i.to_frame().transpose()
    pt_values_i.index = tdata.index

In [ ]:
training_array = pd.concat([tdata, pt_values_i], axis=1).to_crs(4326)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
        "time",
        "fid",
        "index",
        "lulc_class",
        "path",
        "geometry",
        "layer",
        "id",
    ]
)
# Drop rows where there are any NaNs
training_array = training_array.dropna()

training_array.head()

In [ ]:
classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=10,
    n_jobs=-1,
    random_state=42,
)

training_data = np.array(training_array)[:, 1:]
classes = np.array(training_array)[:, 0]

model = classifier.fit(training_data, classes)



In [ ]:
joblib.dump(model, "test_model.dump")


In [ ]:
# filename = 'model_20240209.sav'
# pickle.dump(model, open(filename, 'wb'))
# model = "model_20240209.dump"


In [ ]:
# Print feature importances against column headings
for i in zip(training_array.columns[1:], classifier.feature_importances_):
    print(f"{i[0][0:6]}:\t {i[1]:.2f}")
